### Amazon Cookbooks Topic-modeling: Create Categories and Scarpe for Pics

In [18]:
import numpy as np
import pandas as pd
import re, string
import os, glob, re

from keras.preprocessing import image
from keras.models import Model

#import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from langdetect import detect
from textblob import TextBlob

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, LatentDirichletAllocation, TruncatedSVD, NMF
from sklearn.cluster import KMeans

from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as vgg16_ppi
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as vgg19_ppi
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.resnet_v2 import preprocess_input as Res50V2_ppi
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inceptv3_ppi
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input as moblv2_ppi

from joblib import dump, load
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

    
# --- Parser for reading in the Amazon json files (can be used for both reviews and metadata)
# --- credits folloing parse() method to Julian McAuley UCSD: http://jmcauley.ucsd.edu/data/amazon/ 
def parse(path):
    g = open(path, 'r')
    for l in g:
        yield json.loads(l)
        
PRETRAINED_MODELS = {
    'VGG16': {
        'model': VGG16,
        'preprocess': vgg16_ppi,
        'shape': (224, 224)
    },
    'VGG19': {
        'model': VGG19,
        'preprocess': vgg19_ppi,
        'shape': (224, 224)
    },
    'ResNet50V2': {
        'model': ResNet50V2,
        'preprocess': Res50V2_ppi,
        'shape': (224, 224)
    },
    'InceptionV3': {
        'model': InceptionV3,
        'preprocess': inceptv3_ppi,
        'shape': (299, 299)
    },
    'MobileNetV2': {
        'model': MobileNetV2,
        'preprocess': moblv2_ppi,
        'shape': (224, 224)
    }
}

In [48]:
class amazonCookbookCovers:
    """
    
    
    """
    
    def __init__(self, loc="../data/amzn/covers/"):
        
        self.cookbook_covers_location = loc
        self.getASINs()
        
        # --- for future
        #  Code to check if the 'loc' is a valid dataset
        #  if not, download data and unzip
        #  same for the labels matlab file
    
    def getASINs(self):
        self.asins_with_covers = [f for f in os.listdir(self.cookbook_covers_location) if os.path.isfile(os.path.join(self.cookbook_covers_location, f)) and ".jpg" in f]
    
    def getNextCover(self):
        #count = len(self.asins_with_covers)
        for f in self.asins_with_covers:
            print(f)
            yield f
            
    def getNextNCovers(self, limit):
        count = 0
        Nimages = []
        for f in self.asins_with_covers:
            if count%limit==0 and count!=0:
                yield Nimages
                count = 0
                Nimages = []
            else:
                Nimages.append(f)
                count += 1
        yield Nimages #for the last bunch that might be less than limit
        
        
    def getOneImageArray(self, fileName, target_size=(224, 224)):
        img_path =  self.cookbook_covers_location + fileName
        img = image.load_img(img_path, target_size=target_size)
        x = image.img_to_array(img)
        #x = np.expand_dims(x, axis=0)
        
        return x
       
    def getNextNImageArrays(self, limit=20, target_size=(224, 224) ):
        count = 0
        NimageArrays = []
        for f in self.asins_with_covers:
            if count%limit==0 and count!=0:
                yield np.array(NimageArrays)
                count = 0
                NimageArrays = []
            else:
                NimageArrays.append(self.getOneImageArray(f, target_size=target_size))
                count += 1
        yield np.array(NimageArrays) #for the last bunch that might be less than limit

amzn_cvrs = amazonCookbookCovers()
#print(amzn_cvrs.getNImageArrays(limit=20, target_size=(299, 299)).shape)


In [50]:
a = amazonCookbookCovers().getNextNImageArrays(10,target_size=(299, 299))

[[[[109. 112. 121.]
   [112. 133. 136.]
   [ 86. 133. 127.]
   ...
   [109. 124. 121.]
   [105. 115. 114.]
   [119. 125. 125.]]

  [[ 81. 138. 131.]
   [ 99. 175. 162.]
   [ 87. 191. 168.]
   ...
   [ 40.  56.  53.]
   [ 44.  56.  52.]
   [104. 115. 111.]]

  [[ 77. 138. 130.]
   [ 81. 164. 148.]
   [ 74. 182. 159.]
   ...
   [ 34.  53.  47.]
   [ 30.  45.  40.]
   [108. 120. 116.]]

  ...

  [[107. 126. 120.]
   [ 27.  48.  43.]
   [ 25.  47.  44.]
   ...
   [ 19.  28.  27.]
   [ 26.  35.  32.]
   [107. 116. 113.]]

  [[109. 121. 117.]
   [ 32.  47.  44.]
   [ 31.  47.  46.]
   ...
   [ 14.  20.  20.]
   [ 23.  32.  29.]
   [112. 121. 118.]]

  [[128. 134. 134.]
   [100. 106. 106.]
   [110. 116. 116.]
   ...
   [112. 116. 115.]
   [104. 110. 108.]
   [125. 134. 131.]]]


 [[[ 93.  88.  92.]
   [ 86.  81.  85.]
   [ 78.  73.  77.]
   ...
   [ 56.  68.  80.]
   [ 57.  69.  81.]
   [ 52.  64.  76.]]

  [[ 69.  64.  68.]
   [ 71.  66.  70.]
   [ 73.  68.  74.]
   ...
   [ 52.  64.  76.]
 